In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!pip install evaluate
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00


In [81]:
import torch
from torch.utils import data
from sklearn import model_selection, utils
import evaluate
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

In [82]:
from tqdm.auto import tqdm, trange
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, accuracy_score, f1_score
from IPython.display import display

In [61]:
df_poems = pd.read_csv('poems_dataset.csv')
df_poems

,author,epoch,title,part,text
0,Агнивцев,серебряный век,брат антонио,0,"В монастырской тихой келье,\nПозабывши о весел..."
1,Агнивцев,серебряный век,грузовик 1317,0,"Весь машинный свой век, каждый день по утрам\n..."
2,Агнивцев,серебряный век,грузовик 1317,1,"Грузовик № 1317.\nНо открылись фронты! О, услы..."
3,Агнивцев,серебряный век,мэри пикфорд,0,"В Америке где-то\nСудя по газетам,\nЕсть город..."
4,Агнивцев,серебряный век,бильбокэ,0,"К дофину Франции, в печали,\nСкользнув тайком,..."
...,...,...,...,...,...
54232,Яшин,соцреализм,зеркальце,2,"В руки зеркальце взяла\nИ сказала:\n«Удружи,\n..."
54233,Яшин,соцреализм,очень много солнечного света,0,"Очень много солнечного света,\nНад землей стои..."
54234,Яшин,соцреализм,огонек,0,"Светлячок во мгле —\nОгонек в лесах.\nМожет, о..."
54235,Яшин,соцреализм,назови меня именем светлым,0,"Назови меня именем светлым,\nЧистым именем наз..."


In [62]:
np.random.seed(101)
df_train, df_test = model_selection.train_test_split(df_poems, test_size=0.3)

In [65]:
df_train_balanced = pd.concat([
    df_train[df_train['epoch'] == 'золотой век'].sample(4000),
    df_train[df_train['epoch'] == 'классицизм'],
    df_train[df_train['epoch'] == 'критический реализм'].sample(4000),
    df_train[df_train['epoch'] == 'серебряный век'].sample(4000),
    df_train[df_train['epoch'] == 'соцреализм'].sample(4000),
    df_train[df_train['epoch'] == 'футуризм'].sample(3500),
    df_train[df_train['epoch'] == 'шестидесятники'].sample(3500)
])
df_train_balanced = utils.shuffle(df_train_balanced)

In [66]:
df_labels = pd.concat([
    df_train.groupby('epoch').size(),
    df_train_balanced.groupby('epoch').size(),
    df_test.groupby('epoch').size(),
], axis=1)
df_labels.columns = ['train', 'train_balanced', 'test']
df_labels

,train,train_balanced,test
epoch,,,
золотой век,6009,4000,2552
классицизм,2294,2294,987
критический реализм,6617,4000,2811
серебряный век,11278,4000,4926
соцреализм,4131,4000,1752
футуризм,3818,3500,1630
шестидесятники,3818,3500,1614


In [67]:
datasets = DatasetDict({
    'train': Dataset.from_pandas(df_train_balanced[['text', 'epoch']]),
    'test': Dataset.from_pandas(df_test[['text', 'epoch']])
})

In [68]:
class LabelEncoder:
    def __init__(self, labels):
        self._index_to_label = np.array(labels)
        self._label_to_index = pd.DataFrame(np.arange(len(labels)), index=labels)

    def encode(self, arr_labels):
        return self._label_to_index.loc[arr_labels].to_numpy()

    def decode(self, arr_indices):
        return self._index_to_label[arr_indices]

In [69]:
label_encoder = LabelEncoder(df_poems['epoch'].unique())

In [70]:
datasets_tokenized = datasets.map(
    lambda x: tokenizer(x['text'], truncation=True),
    batched=True,
    remove_columns=['text']
).map(
    lambda x: {'label': label_encoder.encode(x['epoch'])},
    batched=True,
    remove_columns=['epoch']
)

Map:   0%|          | 0/25294 [00:00<?, ? examples/s]

Map:   0%|          | 0/16272 [00:00<?, ? examples/s]

Map:   0%|          | 0/25294 [00:00<?, ? examples/s]

Map:   0%|          | 0/16272 [00:00<?, ? examples/s]

In [72]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2", num_labels=7)
model.cuda()  # uncomment it if you have a GPU

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-

In [73]:
def evaluate_model(model, dev_dataloader, verbose=False, labels=None):
    facts, preds = predict_with_model(model, dev_dataloader)
    pfrs, aucs = get_classification_report(facts, preds, labels)
    if verbose:
        display(pfrs)
        print('aucs:', aucs, np.mean(aucs))
    return np.mean(aucs)

def predict_with_model(model, dataloader):
    preds = []
    facts = []

    for batch in tqdm(dataloader):
        facts.append(batch.labels.cpu().numpy())
        batch = batch.to(model.device)
        with torch.no_grad():
            pr = model(input_ids=batch.input_ids, attention_mask=batch.attention_mask, token_type_ids=batch.token_type_ids)
        preds.append(torch.softmax(pr.logits, -1).cpu().numpy())
    facts = np.concatenate(facts)
    preds = np.concatenate(preds)
    return facts, preds

def get_classification_report(facts, preds, labels=None):
    pfrs = pd.DataFrame(dict(zip(['p', 'r', 'f', 's'], precision_recall_fscore_support(facts, preds.argmax(1)))))
    aucs = [roc_auc_score(facts==i, preds[:, i]) for i in np.unique(facts)]
    pfrs['a'] = aucs
    pfrs = pd.concat([
        pfrs,
        pd.DataFrame(pfrs.mean(axis=0), columns=['mean']).T
    ])
    if labels is not None:
        pfrs.index = list(labels) + ['mean']
    return pfrs, aucs

In [74]:
data_collator = DataCollatorWithPadding(tokenizer)
loader_train = data.DataLoader(
    datasets_tokenized['train'],
    batch_size=64, drop_last=False, shuffle=True, num_workers=0, collate_fn=data_collator
)
loader_test = data.DataLoader(
    datasets_tokenized['test'],
    batch_size=64, drop_last=False, shuffle=True, num_workers=0, collate_fn=data_collator
)

In [75]:
evaluate_model(model, loader_test, verbose=True)

  0%|          | 0/255 [00:00<?, ?it/s]

,p,r,f,s,a
0,0.349911,0.079984,0.130205,4926.000000,0.554200
1,0.116279,0.002854,0.005571,1752.000000,0.482914
2,0.111672,0.205700,0.144757,1614.000000,0.522755
3,0.000000,0.000000,0.000000,2811.000000,0.463881
4,0.068313,0.492025,0.119970,1630.000000,0.310972
5,0.100000,0.010188,0.018492,2552.000000,0.457524
6,0.038095,0.004053,0.007326,987.000000,0.439714
mean,0.112039,0.113543,0.060903,2324.571429,0.461708


aucs: [np.float64(0.5542000096045123), np.float64(0.48291395823112815), np.float64(0.5227545535102441), np.float64(0.4638805290992958), np.float64(0.3109721760160493), np.float64(0.45752385369732307), np.float64(0.43971359435832325)] 0.4617083820738394


np.float64(0.4617083820738394)

In [76]:
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [77]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)

In [78]:
gradient_accumulation_steps = 1
window = 10
cleanup_step = 100
report_step = 10000

In [79]:
ewm_loss = 0
model.train()
cleanup()

for epoch in trange(9):
    tq = tqdm(loader_train)

    for i, batch in enumerate(tq):
        try:
            batch = batch.to(model.device)
            output = model(
                input_ids=batch.input_ids,
                attention_mask=batch.attention_mask,
                token_type_ids=batch.token_type_ids,
                labels=batch.labels
            )
            loss = output.loss
            loss.backward()
        except RuntimeError as e:
            print('error on step', i, e)
            loss = None
            cleanup()
            continue

        if i and i % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        if i % cleanup_step == 0:
            cleanup()

        w = 1 / min(i+1, window)
        ewm_loss = ewm_loss * (1-w) + loss.item() * w
        tq.set_description(f'loss: {ewm_loss:4.4f}')

        if i % report_step == 0:
            model.eval()
            eval_loss = evaluate_model(model, loader_test, verbose=True)
            model.train()
            print(f'epoch {epoch}, step {i}: train loss: {ewm_loss:4.4f}  val auc: {eval_loss}')

model.eval()
eval_loss = evaluate_model(model, loader_test, verbose=True)
print(f'epoch {epoch + 1}, step {i}: train loss: {ewm_loss:4.4f}  val auc: {eval_loss}')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/255 [00:00<?, ?it/s]

,p,r,f,s,a
0,0.349911,0.079984,0.130205,4926.000000,0.554200
1,0.116279,0.002854,0.005571,1752.000000,0.482914
2,0.111672,0.205700,0.144757,1614.000000,0.522755
3,0.000000,0.000000,0.000000,2811.000000,0.463881
4,0.068313,0.492025,0.119970,1630.000000,0.310972
5,0.100000,0.010188,0.018492,2552.000000,0.457524
6,0.038095,0.004053,0.007326,987.000000,0.439714
mean,0.112039,0.113543,0.060903,2324.571429,0.461708


aucs: [np.float64(0.5542000096045123), np.float64(0.48291395823112815), np.float64(0.5227545535102441), np.float64(0.4638805290992958), np.float64(0.3109721760160493), np.float64(0.45752385369732307), np.float64(0.43971359435832325)] 0.4617083820738394
epoch 0, step 0: train loss: 1.9466  val auc: 0.4617083820738394


...

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/255 [00:00<?, ?it/s]

,p,r,f,s,a
0,0.735319,0.701583,0.718055,4926.000000,0.897310
1,0.528172,0.615297,0.568416,1752.000000,0.919891
2,0.515575,0.594796,0.552359,1614.000000,0.906364
3,0.744248,0.494842,0.594444,2811.000000,0.899743
4,0.803787,0.859509,0.830714,1630.000000,0.973835
5,0.604651,0.672414,0.636735,2552.000000,0.913011
6,0.696473,0.860182,0.769719,987.000000,0.985960
mean,0.661175,0.685518,0.667206,2324.571429,0.928016


aucs: [np.float64(0.897310004745717), np.float64(0.9198910807955017), np.float64(0.906363603163275), np.float64(0.8997425689577261), np.float64(0.9738349549954202), np.float64(0.9130111465768573), np.float64(0.9859604362767663)] 0.9280162565016091
epoch 9, step 0: train loss: 0.5957  val auc: 0.9280162565016091


KeyboardInterrupt: 

## Testing

In [86]:
labels, probs = predict_with_model(model, loader_test)
predictions = probs.argmax(axis=1)

  0%|          | 0/255 [00:00<?, ?it/s]

In [88]:
accuracy = accuracy_score(labels, predictions)
f1 = f1_score(labels, predictions, average='weighted')
print('Accuracy: {:.4f}'.format(accuracy))
print('F1: {:.4f}'.format(f1))

Accuracy: 0.6472
F1: 0.6504


In [89]:
model.save_pretrained('rubert-tiny-poems')
tokenizer.save_pretrained('rubert-tiny-poems')

('rubert-tiny-poems/tokenizer_config.json',
 'rubert-tiny-poems/special_tokens_map.json',
 'rubert-tiny-poems/vocab.txt',
 'rubert-tiny-poems/added_tokens.json',
 'rubert-tiny-poems/tokenizer.json')

In [90]:
!zip -r rubert-tiny-poems.zip rubert-tiny-poems

  adding: rubert-tiny-poems/ (stored 0%)
  adding: rubert-tiny-poems/config.json (deflated 55%)
  adding: rubert-tiny-poems/tokenizer_config.json (deflated 73%)
  adding: rubert-tiny-poems/special_tokens_map.json (deflated 80%)
  adding: rubert-tiny-poems/vocab.txt (deflated 64%)
  adding: rubert-tiny-poems/tokenizer.json (deflated 73%)
  adding: rubert-tiny-poems/model.safetensors (deflated 8%)


## Использование

In [93]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

use_tokenizer = AutoTokenizer.from_pretrained('rubert-tiny-poems')
use_model = AutoModelForSequenceClassification.from_pretrained('rubert-tiny-poems')
if torch.cuda.is_available():
    use_model.cuda()

In [95]:
def get_poem_epoch(text):
    with torch.no_grad():
        inputs = use_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(use_model.device)
        probs = torch.sigmoid(use_model(**inputs).logits).cpu().numpy()[0]
    return label_encoder.decode(probs.argmax())

In [117]:
# Волошин, "Таиах", Серебряный век
get_poem_epoch("""
Они проходят по земле
Слепые и глухонемые
И чертят знаки огневые
В распахивающейся мгле.

Собою бездны озаряя,
Они не видят ничего,
Они творят, не постигая
Предназначенья своего.

Сквозь дымный сумрак преисподней
Они кидают вещий луч...
Их судьбы – это лик Господний,
Во мраке явленный из туч.
""")

'серебряный век'

In [118]:
# Маяковский, "Стихи о советском паспорте", футуризм
# (нарочно без "лесенки")
get_poem_epoch("""
И вдруг, как будто ожогом, рот
скривило господину.
Это господин чиновник берет
мою краснокожую паспортину.
Берет — как бомбу, берет — как ежа,
как бритву обоюдоострую,
берет, как гремучую в 20 жал
змею двухметроворостую.
Моргнул многозначаще глаз носильщика,
хоть вещи снесет задаром вам.
Жандарм вопросительно смотрит на сыщика,
сыщик на жандарма.
С каким наслажденьем жандармской кастой
я был бы исхлестан и распят
за то, что в руках у меня молоткастый,
серпастый советский паспорт.
Я волком бы выгрыз бюрократизм.
К мандатам почтения нету.
К любым чертям с матерями катись
любая бумажка. Но эту…
Я достаю из широких штанин
дубликатом бесценного груза.
Читайте, завидуйте, я — гражданин
Советского Союза.
""")

'футуризм'

In [119]:
# Симонов, "Жди меня", соцреализм
get_poem_epoch("""
Жди меня, и я вернусь,
Не желай добра
Всем, кто знает наизусть,
Что забыть пора.
Пусть поверят сын и мать
В то, что нет меня,
Пусть друзья устанут ждать,
Сядут у огня,
Выпьют горькое вино
На помин души…
Жди. И с ними заодно
Выпить не спеши.
Жди меня, и я вернусь,
Всем смертям назло.
Кто не ждал меня, тот пусть
Скажет: — Повезло.
Не понять, не ждавшим им,
Как среди огня
Ожиданием своим
Ты спасла меня.
Как я выжил, будем знать
Только мы с тобой, —
Просто ты умела ждать,
Как никто другой.
""")

'серебряный век'

In [120]:
# Твардовский, "Василий Теркин" (8. Гармонь), соцреализм
get_poem_epoch("""
По дороге прифронтовой,
Запоясан, как в строю,
Шел боец в шинели новой,
Догонял свой полк стрелковый,
Роту первую свою.
Шел легко и даже браво
По причине по такой,
Что махал своею правой,
Как и левою рукой.
Отлежался. Да к тому же
Щелкал по лесу мороз,
Защемлял в пути все туже,
Подгонял, под мышки нес.
Вдруг — сигнал за поворотом,
Дверцу выбросил шофер,
Тормозит:
— Садись, пехота,
Щеки снегом бы натер.
""")

'соцреализм'